# Funkcijsko programiranje 2

## Vračanje funkcij

Funkcije višjega reda sprejemajo in/ali vračajo funkcije. Prvo že poznamo, sedaj pa si oglejmo še primer funkcije, ki vrne neko funkcijo. Napišimo funkcijo `aritmetika`, ki bo vrnila funkcijo, ki bo predstavljala eno od aritmetičnih operacij, npr. množenje. To vrnjeno funkcijo lahko definiramo znotraj funkcije aritmetika, lahko uporabimo lambda funkcijo, lahko pa vrnemo kakšno že obstoječo funkcijo, npr. `mul`.

In [1]:
def aritmetika():
    def zmnozi(a, b):
        return a * b
    return zmnozi

def aritmetika():
    return lambda a,b: a*b

from operator import mul
def aritmetika():
    return mul

f = aritmetika()
print(f(2,7))

Sedaj pa napišimo funkcijo `mnozilnik`, ki bo sprejela argument `k` in vrnila funkcijo, ki bo znala množiti s `k`. Ponovno lahko definiramo to novo funkcijo znotraj funkcije ali pa uporabimo anonimno/lambda funkcijo

In [2]:
def mnozilnik(k):
    def mnozi(x):
        return k*x
    return mnozi

def mnozilnik(k):
    return lambda x: k*x

mul3 = mnozilnik(3)
print(mul3(7), mnozilnik(2)(5))

Pripravimo si lahko celo tabelo funkcij, kjer se na i-tem mestu nahaja množilnik z i.

In [3]:
mnozilniki = [mnozilnik(k) for k in range(10)]
mul2 = mnozilniki[2]
print(mul2(4), mnozilniki[3](5))

8 15


Napišimo funkcijo `izmeri`, ki bo sprejela neko funkcijo `f`, jo izvedla, vrnila njen rezultat, ob tem pa bo izmerila in izpisala čas izvajanja funkcije. Taki funkciji bi rekli ovojnica oz. `wrapper`. Posplošimo jo tako, da bomo lahko ovojnici poleg funkcije podali tudi parametre te funkcije.

In [4]:
from time import time

def vsota_kvadratov(n=10**6):
    return sum(i**2 for i in range(n))

def izmeri(f, *args, **kwargs):
    start = time()
    ret = f(*args, **kwargs)
    end = time()
    print("[izmeri] Čas izvajanja:", end - start)
    return ret

print(izmeri(vsota_kvadratov))
print(izmeri(vsota_kvadratov, 3*10**6))

[izmeri] Čas izvajanja: 0.2284679412841797
333332833333500000
[izmeri] Čas izvajanja: 0.6539406776428223
8999995500000500000


## Dekoratorji

Dekoratorji so funkcije, ki sprejmejo drugo funkcijo, razširijo njeno funkcionalnost in vrnejo novo spremenjeno funkcijo. Prejšnja ovojnica za merjenje časa je ob klicu takoj izvedla podano funkcijo in sporočila izmerjen čas. Dekorator pa podane funkcije ne bo takoj izvedel, ampak bo zgradil in vrnil novo funkcijo, ki bo izpisala svoj čas izvajanja, ko jo bomo uporabili (če jo sploh bomo).

Dekorator `casomerilec` sprejme funkcijo `f` in znotraj definira in vrne novo funkcijo `wrapper`, ki razširja funkcijo `f`.

In [5]:
def casomerilec(f):
    def wrapper():
        start = time()
        rezultat = f()
        end = time()
        print("[dekorator] Čas izvajanja:", end - start)
        return rezultat
    return wrapper

vsota_kvadratov_cas = casomerilec(vsota_kvadratov)
print(vsota_kvadratov_cas())

[dekorator] Čas izvajanja: 0.20917105674743652
333332833333500000


Kot v prejšnjem primeru lahko dekorirano funkcijo ustvarimo z uporabo dekoratorja, ki mu podamo neko funkcijo, ta pa nam vrne dekorirano funkcijo. Druga možnost je, da uporabimo dekorator že kar pri definiciji funkcije s sintakso `@ime_dekoratorja`.

In [6]:
@casomerilec
def vsota_kvadratov():
    return sum(i**2 for i in range(10**5))

def vsota_kubov():
    return sum(i**3 for i in range(10**5))

print("kvadrati:", vsota_kvadratov())
print("kubi:", vsota_kubov())

[dekorator] Čas izvajanja: 0.025547027587890625
kvadrati: 333328333350000
kubi: 24999500002500000000


Dopolnimo naš dekorator, da bo mogoče z njim dekorirati funkcije s poljubnimi argumenti (`*args`, `**kwargs`). Pri pisanju dekoratorja ne smemo pozabiti tudi na vračanje rezultata, ki ga vrne funkcija. Delovanje lahko preizkusimo na funkciji za izračun vsote potenc $\text{vsota_potenc}(p, n) = \sum_{i=0}^n i^{p}$.

In [7]:
def casomerilec(f):
    def wrapper(*args, **kwargs):
        start, ret, end = time(), f(*args,  **kwargs), time()
        print("[dekorator] Čas izvajanja:", end - start)
        return ret
    return wrapper

@casomerilec
def vsota_potenc(p=2, n=10**6):
    return sum(i**p for i in range(n+1))

print(vsota_potenc(1))
print(vsota_potenc(n=10**5))

[dekorator] Čas izvajanja: 0.20885658264160156
500000500000
[dekorator] Čas izvajanja: 0.0156252384185791
333338333350000


Dekorator (funkcija, ki bo vrnila novo funkcijo) lahko tudi sam sprejme kakšen argument. Naš časomerilec lahko spremenimo tako, da bo večkrat pognal funkcijo in izmeril čas, na koncu pa izpisal povprečen čas izvajanja. Število ponovitev pa naj bo argument dekoratorja. Dekorator z argumentom (`casomerilec_veckrat`) mora vrniti navaden dekorator (`casomerilec`). Navaden dekorator pa sprejme funkcijo (`f`) in vrne dekorirano funkcijo (`wrapper`). Vrnjena dekorirana funkcija pa bo sprejela argumente (`*args`, `**kwargs`), ki jih bo posredovala funkciji `f`.

In [8]:
def casomerilec_veckrat(ponovitve):
    def casomerilec(f):
        def wrapper(*args, **kwargs):
            start = time()
            for i in range(ponovitve):
                ret = f(*args,  **kwargs)
            end = time()
            print(f"[dekorator z arg] Povprečen čas izvajanja ({ponovitve} ponovitev): {(end - start)/ponovitve}")
            return ret
        return wrapper
    return casomerilec

@casomerilec_veckrat(10)
def vsota_potenc(p=2, n=10**6):
    return sum(i**p for i in range(n))

print(vsota_potenc(3, n=10**5))

[dekorator z arg] Povprečen čas izvajanja (10 ponovitev): 0.024920964241027833
24999500002500000000


### Memoizacija

Oglejmo si praktičen primer uporabe dekoratorja. Če isto funkcijo pokličemo večkrat z enakimi argumenti, bo vrnila enak rezultat. Zato bi lahko pridobili na učinkovitosti, če si ob prvem klicu funkcije rezultat shranimo. Ob naslednjih klicih z enakimi argumenti pa samo vrnemo rezultat iz shrambe, namesto da bi ponovno izvedli funkcijo. Ta pristop je znan pod imenom memoizacija (angl. *memoization*) (brez "r", izhaja iz besede *memo*). Za primer uporabimo tehniko na računanju Fibonaccijevih števil.

In [9]:
def fib(n):
    return fib(n-2)+fib(n-1) if n>1 else n

print([fib(i) for i in range(10)])

memo = {}
def fib(n):
    if n in memo: return memo[n]  # je rezultat že shranjen?
    rez = fib(n-2)+fib(n-1) if n>1 else n  # izračunaj rezultat
    memo[n] = rez  # shrani rezultat
    return rez

print(fib(100))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
354224848179261915075


Funkcijo smo spremenili tako, da smo na začetek in konec dodali malo logike za shranjevanje rezultatov. Povsem enako bi lahko spremenili tudi kakšne druge funkcije. Namesto popravljanja funkcij vsepovsod lahko napišemo dekorator in z njim elegantno dekoriramo funkcije, za katere želimo, da si shranjujejo rezultate. V modulu `functools` prav tak dekorator `lru_cache` (Least Recently Used) že obstaja (https://docs.python.org/3/library/functools.html#functools.lru_cache). Sprejme celo argument, ki predstavlja največjo velikost shrambe (če je enak `None`, je neomejena).

In [10]:
from functools import lru_cache

@lru_cache(None)
def fib(n):
    return fib(n-2)+fib(n-1) if n>1 else n

print(fib(100))

354224848179261915075


## Delna aplikacija

V delni aplikaciji funkcije podamo funkciji nekatere argumente, pri tem pa nastane nova funkcija, ki je sposobna sprejeti preostanek argumentov. Funkcija `pow` sprejme dva argumenta, osnovo in eksponent. Uporabno bi bilo, če bi lahko funkciji `pow` podali samo osnovo 2 in s tem dobili funkcijo, ki bo sprejela še eksponent in vrnila iskano potenco dvojke. Žal to v Pythonu kar tako ne deluje.

In [11]:
print(pow(2,5))
p2 = pow(2)
print(p2(5), p2(10))

32


TypeError: pow() missing required argument 'exp' (pos 2)

**Currying** je pristop, ki funkcijo z več argumenti pretvori v zaporedje funkcij, ki sprejmejo po en argument. Funkcijo `f` bi tako pretvorili v funkcijo `g`, ki sprejme prvi argument in vrne pretvorjeno funkcijo, ki na enak način sprejme preostale argumente. Klic funkcije `f(3,5,6,8)` bi bil tako enakovreden izrazu `g(3)(5)(6)(8)`.

Oglejmo si najprej dekorator `curry2`, ki bo deloval za dekoriranje funkcij z dvema argumentoma. Dekorator vrne funkcijo `wrapper1`, ki sprejme prvi argument in vrne funkcijo `wrapper2`, ki bo sprejela še drugi argument in vrne izračunan rezultat.

In [12]:
def curry2(f):
    def wrapper1(x):
        def wrapper2(y):
            return f(x,y)
        return wrapper2
    return wrapper1

@curry2
def potenca(osnova, eksponent):
    return pow(osnova, eksponent)

p2 = potenca(2)
print(p2(5))
print(potenca(3)(2))

32
9


Posplošimo dekorator na funkcije s poljubnim številom argumentov. V ta namen bomo prejete argumente zlagali v dekoratorjevo lokalni spremenljivko (seznam) `argumenti`. Vrnjena funkcija `wrapper` bo sprejela en argument in ga dodala v seznam. Če bo dolžina tega seznama enaka številu argumentov, ki jih zahteva dekorirana funkcija `f` (`inspect.signature(f).parameters`), izračuna in vrne rezultat funkcije `f`. Sicer pa vrne isto funkcijo, ki bo sprejela in dodala v seznam naslednji argument.

In [13]:
from inspect import signature

def curry(f):
    argumenti = []
    def wrapper(x):
        argumenti.append(x)
        if len(argumenti) == len(signature(f).parameters):
            return f(*argumenti)
        else:
            return wrapper
    return wrapper

curry_potenca = curry(pow)  # pow(base, exp[, mod])
print(curry_potenca(3)(4)(100))
print(curry_potenca(3)(4)(10))  # ne deluje

81
<function curry.<locals>.wrapper at 0x00000209D2EE8D30>


Druga uporaba dekorirane funkcije ne deluje, ker uporablja isti seznam argumentov, ki je napolnjen že od prej. Pred vsako uporabo bi morali funkcijo dekorirati znova. Težavo bi lahko rešili, če bi si klici funkcije `wrapper` podajali seznam argumentov, namesto da polnijo isti seznam.

Dekorator `curry` naj vrne funkcijo `akumulator`, ki bo sprejela seznam podanih argumentov `*args` in vrnila rezultat (če je argumentov dovolj) ali pa (anonimno) funkcijo, ki sprejme dodatne argumente `*other`. Te dodatne argumente želimo združiti z obstoječimi in jih podati funkciji `akumulator`. Ta bo torej prejela podaljšan seznam argumentov in vrnila rezultat ali pa čakala na nadaljnje argumente. Rešitev je dovolj splošna, da sprejme tudi po več kot en argument hkrati.

In [14]:
def curry(f):
    def akumulator(*args):
        if len(args) == len(signature(f).parameters):
            return f(*args)
        else:
            return lambda *other: akumulator(*(args + other))
    return akumulator

curry_potenca = curry(pow)
print(curry_potenca(3)(4)(100))
print(curry_potenca(3)(4)(10))
print(curry_potenca(3,4)()(70))

81
1
11


Python ponuja podobno funkcionalnost **delne aplikacije** s podajanjem nekaterih argumentov (ne nujno začetnih). Na voljo je funkcija `functools.partial`, ki sprejme funkcijo in nekaj njenih argumentov, ter vrne specializirano funkcijo, ki ima podane argumente fiksirane, sprejme pa preostanek argumentov in vrne rezultat.

In [15]:
from functools import partial

p2 = partial(pow, 2)
print("pow2", p2(10))

cube = partial(pow, exp=3)
print("cube", cube(10))

pow2 1024
cube 1000


Nobenega razloga ni, da ne bi bili tudi podani argumenti funkcije. Tako lahko npr. iz funkcij `map` in `abs` enostavno sestavimo funkcijo, ki bo izračunala absolutne vrednosti podanega seznama.

In [16]:
map_abs = partial(map, abs)
print(list(map_abs([3,-7,8,-1])))

[3, 7, 8, 1]
